In [ ]:
#Tarun Kumar Alapati

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

In [ ]:
#import libraries
import sys
import re 
from pyspark import SparkContext, SparkConf
from collections import Counter

In [ ]:
SUPPORT = 85
CONFIDENCE = 0.9

In [ ]:
# create Spark context with necessary configuration
sc = SparkContext("local","PySpark Word Count Exmaple")

In [ ]:
#File Path
file="/content/drive/My Drive/Big Data Assignments/Project1/browsing.txt"

In [ ]:
#Read data into rdd
singletons={}
def filterspaces(line):
    for word in line:
        if word == '':
          line.remove(word)
    return line   

fileRDD = sc.textFile(file)
wordsRDD = fileRDD.map(lambda line: line.split(" ")).map(lambda line: filterspaces(line)).map(lambda line: frozenset(line))
wordslist= fileRDD.map(lambda line: line.split(" ")).map(lambda line: filterspaces(line))

In [ ]:
#Broadcast all the itemsets to all executer nodes
shared_itemset = sc.broadcast(wordsRDD.map(lambda x: frozenset(x)).collect())

In [ ]:
shared_itemset.value

In [ ]:
#Generate singleton file
singletons = Counter()
for basket in shared_itemset.value:
  singletons.update(basket)
  #considering them only if the total count >= support
  candidateSingletons = []
  candidateDictSingletons = {}
  finalDict = {}
  for x in singletons:
    if singletons[x] >= SUPPORT:
      candidateSingletons.append(frozenset([x]))
      finalDict[frozenset([x])] = singletons[x]
    #candidateDictSingletons[frozenset([x])] = singletons[x]
    
  #Sorting the frequent items
  frequentSingletons = sorted(candidateSingletons)
  frequentItems = set(frequentSingletons)

In [ ]:
#Generating Candidate Sets of frequent items
def getItemsforK(frequentItems, k):
    frequentItems = list(frequentItems)
    combs = []
    for i, item1 in enumerate(frequentItems):
      for item2 in frequentItems[i+1:]:
        if(list(item1)[:k - 2] == list(item2)[:k - 2]):
          combs.append(item1 | item2)
    return combs

In [ ]:
#Generating frequent itemsets with support greater than 85
def getFrequentItems(frequentitemset, support, shared_itemset):
  def getSupport(x):
    x_sup = len([1 for t in shared_itemset.value if x.issubset(t)])
    if x_sup >= support:
      return x, x_sup
    else:
      return ()
  frequentitemsK = sc.parallelize(frequentitemset).map(getSupport).filter(lambda x: x).collect()
  return frequentitemsK

In [ ]:
#Apriori Algorithm that generates all the rules by checking the support and return a dictionary with final set of frequent items

def apriori(frequentItems, support,finalDict):
  k=2
  while k <= 4:
    candidateFrequentItems = getItemsforK(frequentItems, k)
    newFrequentItems = getFrequentItems(candidateFrequentItems,support,shared_itemset)
    #finalResult.extend(newFrequentItems)
    for item, support in newFrequentItems:
      finalDict[item] = support
    frequentItems = set(x[0] for x in newFrequentItems)
    #frequentItems.sort()
    k=k+1

  #for item, support in frequentItems:
    #finalResult[item] = support
    
  #finalResult.extend(frequentItems)
  #return finalResult
  return finalDict

In [ ]:
dictA = apriori(frequentItems,SUPPORT,finalDict)

In [ ]:
broadcast_frequentItems = sc.broadcast(dictA)

In [ ]:
#Function to get the confidence of each of the rules
def getConfidence(left, right, broadcast_frequentItems):
  confidence = (broadcast_frequentItems[frozenset(left | right)])/broadcast_frequentItems[frozenset(left)]
  return confidence

In [ ]:
#Generates the rules into left and right item sets
def getRules(candidateKey, minedfrequentItemSets):
  rules = []
  #print(candidateKey)
  for item in candidateKey:
    right = set([item])
    #print(right)
    left = set(candidateKey)
    left.remove(item)
    #print(left)
    confidence = getConfidence(left,right, minedfrequentItemSets)
    if confidence >= CONFIDENCE:
      rules.append((",".join(left)+"->"+item,confidence))
      #print(rules)
  return rules

In [ ]:
# Considers frequent sets of size > 1
RDD1 = sc.parallelize(dictA.items()).filter(lambda x : len(x[0]) > 1)
RDD1.collect()

In [ ]:
RDD2 = RDD1.flatMap(lambda x : getRules(x[0], broadcast_frequentItems.value))
RDD2.collect()

In [ ]:
#Generates Ruels along with the confidence and output into a file
RDD3 = RDD2.sortBy(lambda x : x[1], False).map(lambda x : x[0] + "; Confidence=" + str(round(x[1]*100, 2)) + "%").coalesce(1).saveAsTextFile("/content/associationRulesWithSupport")

In [ ]:
!cat /content/associationRulesWithSupport/part-00000